In [1]:
import pandas as pd 
import glob
import cv2 as cv
import random
import os

import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import PIL.ImageOps    

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.utils
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.metrics import f1_score


from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler
from tqdm.auto import tqdm
import timm
import math
from sklearn.model_selection import train_test_split



/opt/conda/envs/SAM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Make DataFrame

## columns: review_img_path, product_img_path, label
---

In [2]:
all_product_img_list = glob.glob('./masked_data/product_img/*')
all_product_img_list = sorted(all_product_img_list)
# print(all_product_img_list)

In [3]:
review_img_path = []
product_img_path = []
label = []
for product_img in all_product_img_list:
    glob_img_list = []
    str = product_img.split("/")[3]
    str = str.split(".")[0]
    glob_img_list = glob.glob('./masked_data/review_img/'+str +'_review_img/O/*.jpg')
    review_img_path = review_img_path + glob_img_list
    product_img_path = product_img_path + [product_img  for i in range(len(glob_img_list))]

In [4]:
print(len(product_img_path))
print(len(review_img_path))


5389
5389


In [5]:
for i in range(len(review_img_path)):
    random_number = random.randint(0, 1)
    if random_number == 0:
        label.append(0)
    else:
        review_num = review_img_path[i].split('/')[3]
        review_num = review_num.split("_")[0]
        random_number = random.randint(0, len(all_product_img_list) - 1)
        while review_num == random_number:
            random_number = random.randint(0, len(all_product_img_list) - 1)
        product_img_path[i] = all_product_img_list[random_number]
        label.append(1)

In [6]:
df = pd.DataFrame(columns=['review_img_path','product_img_path', 'label'])
df['review_img_path'] = review_img_path
df['product_img_path'] = product_img_path
df['label'] = label

df

,review_img_path,product_img_path,label
0,./masked_data/review_img/0_review_img/O/404.jpg,./masked_data/product_img/78.jpg,1
1,./masked_data/review_img/0_review_img/O/174.jpg,./masked_data/product_img/0.jpg,1
2,./masked_data/review_img/0_review_img/O/93.jpg,./masked_data/product_img/42.jpg,1
3,./masked_data/review_img/0_review_img/O/177.jpg,./masked_data/product_img/0.jpg,0
4,./masked_data/review_img/0_review_img/O/399.jpg,./masked_data/product_img/39.jpg,1
...,...,...,...
5384,./masked_data/review_img/98_review_img/O/194.jpg,./masked_data/product_img/98.jpg,0
5385,./masked_data/review_img/98_review_img/O/148.jpg,./masked_data/product_img/98.jpg,0
5386,./masked_data/review_img/98_review_img/O/401.jpg,./masked_data/product_img/34.jpg,1
5387,./masked_data/review_img/98_review_img/O/433.jpg,./masked_data/product_img/98.jpg,0


---

# Siames Net

In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [8]:
CFG = {
    'IMG_SIZE':256,
    'EPOCHS':10,
    'LEARNING_RATE':3e-2,
    'BATCH_SIZE':8,
    'SEED':41
}

In [9]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = 'a'
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [10]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])


In [11]:
class SiameseNetworkDataset(Dataset):
    def __init__(self,review_img_path,product_img_path,label,transform=None):
        self.review_img_path = review_img_path
        self.product_img_path = product_img_path
        self.label = label
        self.transform = transform
        
    def __getitem__(self,index):
        review_img = cv.imread(self.review_img_path[index])
        product_img = cv.imread(self.product_img_path[index])
        review_img = cv.resize(review_img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
        product_img = cv.resize(product_img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
        image = cv.hconcat([review_img, product_img])

        if self.transform is not None:
            image  = self.transform(image=image)['image']
    
        return image, torch.from_numpy(np.array([int(label[index])], dtype=np.float32))
    
    def __len__(self):
        return len(self.review_img_path)

In [12]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            ToTensorV2()
                            ])


In [13]:
train_dataset = SiameseNetworkDataset(train["review_img_path"].values, train["product_img_path"].values, train["label"].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = SiameseNetworkDataset(val["review_img_path"].values, val["product_img_path"].values, val["label"].values, train_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [14]:
avail_pretrained_models = timm.list_models(pretrained=True)
print(avail_pretrained_models)

['bat_resnext26ts.ch_in1k', 'beit_base_patch16_224.in22k_ft_in22k', 'beit_base_patch16_224.in22k_ft_in22k_in1k', 'beit_base_patch16_384.in22k_ft_in22k_in1k', 'beit_large_patch16_224.in22k_ft_in22k', 'beit_large_patch16_224.in22k_ft_in22k_in1k', 'beit_large_patch16_384.in22k_ft_in22k_in1k', 'beit_large_patch16_512.in22k_ft_in22k_in1k', 'beitv2_base_patch16_224.in1k_ft_in1k', 'beitv2_base_patch16_224.in1k_ft_in22k', 'beitv2_base_patch16_224.in1k_ft_in22k_in1k', 'beitv2_large_patch16_224.in1k_ft_in1k', 'beitv2_large_patch16_224.in1k_ft_in22k', 'beitv2_large_patch16_224.in1k_ft_in22k_in1k', 'botnet26t_256.c1_in1k', 'caformer_b36.sail_in1k', 'caformer_b36.sail_in1k_384', 'caformer_b36.sail_in22k', 'caformer_b36.sail_in22k_ft_in1k', 'caformer_b36.sail_in22k_ft_in1k_384', 'caformer_m36.sail_in1k', 'caformer_m36.sail_in1k_384', 'caformer_m36.sail_in22k', 'caformer_m36.sail_in22k_ft_in1k', 'caformer_m36.sail_in22k_ft_in1k_384', 'caformer_s18.sail_in1k', 'caformer_s18.sail_in1k_384', 'caformer_s

In [15]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        # self.backbone = timm.create_model('maxvit_tiny_tf_512', pretrained=True)
        self.backbone = timm.create_model('efficientnet_b0', pretrained=False)
        self.classifier1 = nn.Linear(1000, 10)
        self.classifier2 = nn.Linear(10, 1)
        
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier1(x)
        x = self.classifier2(x)
        return x

In [16]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []
    correct = 0
    total = 0

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            val_loss.append(loss.item())
            _, predicted = torch.max(pred.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
        
        _val_loss = np.mean(val_loss)
        # _val_score = f1_score(true_labels, preds, average='weighted')
        _val_score = 100 * correct // total
        
    
    return _val_loss, _val_score

In [17]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    # criterion = nn.BCEWithLogitsLoss.to(device)
    criterion = nn.BCEWithLogitsLoss(weight=None, size_average=None, reduce=None, reduction='mean', pos_weight=None)
    scaler = GradScaler()
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            with autocast():
                output = model(imgs)
                loss = criterion(output, labels)
            
            # loss.backward()
            # optimizer.step()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

In [18]:
model = BaseModel()
model.eval()

optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

100%|██████████| 203/203 [00:09<00:00, 21.21it/s]


Epoch [1], Train Loss : [2.41412] Val Loss : [0.71207] Val Weighted F1 Score : [3255.00000]


100%|██████████| 203/203 [00:09<00:00, 20.91it/s]


Epoch [2], Train Loss : [0.70212] Val Loss : [0.69233] Val Weighted F1 Score : [3255.00000]


100%|██████████| 203/203 [00:09<00:00, 21.98it/s]


...

Epoch [5], Train Loss : [0.69410] Val Loss : [0.70107] Val Weighted F1 Score : [3255.00000]


100%|██████████| 203/203 [00:09<00:00, 21.57it/s]


Epoch [6], Train Loss : [0.69426] Val Loss : [0.69525] Val Weighted F1 Score : [3255.00000]


 45%|████▌     | 92/203 [00:04<00:05, 22.01it/s]


KeyboardInterrupt: 

In [ ]:
# # Locate the test dataset and load it into the SiameseNetworkDataset
# folder_dataset_test = datasets.ImageFolder(root="./output/val")
# siamese_dataset = SiameseNetworkDataset(val["review_img_path"].values, 
#                                         val["product_img_path"].values,
#                                         val["label"].values,
#                                         transform=train_transform)
# test_dataloader = DataLoader(siamese_dataset, num_workers=0, batch_size=1, shuffle=True)

# # Grab one image that we are going to test
# dataiter = iter(test_dataloader)
# x0, _, _ = next(dataiter)

# for i in range(10):
#     # Iterate over 10 images and test them with the first image (x0)
#     _, x1, label2 = next(dataiter)

#     # Concatenate the two images together
#     concatenated = torch.cat((x0, x1), 0)
    
#     output1, output2 = model.parameters(x0.cuda(), x1.cuda())
#     euclidean_distance = F.pairwise_distance(output1, output2)
#     imshow(torchvision.utils.make_grid(concatenated), f'Dissimilarity: {euclidean_distance.item():.2f}')